In [ ]:
from importlib import reload
import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

sys.path.append(parent_dir)

import src
reload(src)


from src.utils import load_MNIST_data
from src.model import simple_model
from src.train import train_model
from src.inference import evaluate_overall, evaluate_classwise, confusion_matrix_plot, plot_roc_curves

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt




In [3]:
train_dloader, val_dloader, test_dloader = load_MNIST_data(batch_size = 64)